In [12]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [14]:
# Load dataset
data = pd.read_csv("dataset_1.csv")  # Replace with your file path

In [16]:
# Feature selection
X = data[["Depth", "WOB", "SURF_RPM", "PHIF", "VSH"]]  # Input features
y = data["ROP_AVG"]  # Target feature

In [18]:
# Normalize/Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [20]:
# Train an XGBoost surrogate model
surrogate_model = XGBRegressor(n_estimators=100, random_state=42)
surrogate_model.fit(X_scaled, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [22]:
# Define the objective function for Genetic Algorithm
def objective_function(individual):
    # Scale the input
    X_new = scaler.transform([individual])
    # Predict ROP_AVG using the XGBoost model
    rop = surrogate_model.predict(X_new)
    return rop[0],  # Return the predicted ROP_AVG (as a tuple for DEAP)


In [24]:
# Define parameter bounds
param_bounds = {
    "Depth": (data["Depth"].min(), data["Depth"].max()),
    "WOB": (data["WOB"].min(), data["WOB"].max()),
    "SURF_RPM": (data["SURF_RPM"].min(), data["SURF_RPM"].max()),
    "PHIF": (data["PHIF"].min(), data["PHIF"].max()),
    "VSH": (data["VSH"].min(), data["VSH"].max())
}

In [26]:
# Set up Genetic Algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize ROP_AVG
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

In [28]:
# Define genes (parameters) with their bounds
for i, (key, (low, high)) in enumerate(param_bounds.items()):
    toolbox.register(f"attr_{i}", random.uniform, low, high)

In [30]:
# Create an individual with 5 genes (Depth, WOB, SURF_RPM, PHIF, VSH)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_0, toolbox.attr_1, toolbox.attr_2, toolbox.attr_3, toolbox.attr_4), n=1)

In [32]:
# Create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [34]:
# Register the objective function
toolbox.register("evaluate", objective_function)

In [36]:
# Register genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Blend crossover
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)  # Gaussian mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection

In [38]:
# Create initial population
population = toolbox.population(n=50)  # Population size = 50

In [40]:
# Run the Genetic Algorithm
ngen = 40  # Number of generations
cxpb = 0.7  # Crossover probability
mutpb = 0.2  # Mutation probability

In [42]:
# Run the algorithm
final_pop = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

gen	nevals
0  	50    
1  	32    
2  	42    
3  	35    
4  	38    


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

5  	42    
6  	41    
7  	43    
8  	36    


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

...

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

33 	33    
34 	38    
35 	35    
36 	35    
37 	33    


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

38 	39    
39 	40    
40 	40    


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

In [44]:
# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_rop = objective_function(best_individual)[0]

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [46]:
print("Best Individual (Optimized Parameters):", best_individual)
print("Optimized ROP_AVG:", best_rop)

Best Individual (Optimized Parameters): [3786.2959738161917, 19016.657530064545, 2.565001782686284, 5.506333622414232, 4.309113568916649]
Optimized ROP_AVG: 0.008572021


---

The **magnitude of improvement** on ROP_AVG using **Genetic Algorithm** is calculated as follows:

- **Improvement** = Optimized ROP_AVG (GA) - Baseline ROP_AVG  
  \( 0.008572021 - 0.00779683 = 0.000775191 \)

- **Percentage Improvement** = \( \frac{{\text{Improvement}}}{{\text{Baseline ROP_AVG}}} \times 100 \)  
  \( \frac{{0.000775191}}{{0.00779683}} \times 100 \approx 9.95\% \)

A **9.95% improvement** using the **Genetic Algorithm** is also significant, offering potential **cost savings** and **efficiency improvements** in industrial drilling operations.

Again, to ensure the best results, we will compare this with the performance of **other Optimization techniques**.

---